<a href="https://colab.research.google.com/github/proliuk/ESONERO-SOCIAL-NETWORK-ANALYSIS-GEPHI/blob/master/Definitivo_Esonero_Gephi_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ISTRUZIONI PER COLLEGARE GOOGLE DRIVE
from google.colab import drive
import pandas as pd
from pandas import DataFrame
drive.mount('/content/drive', force_remount="True")
#IMPORTO I VARI FILE NECESSARI 
path = '/content/drive/My Drive/gephi/combined_csv.csv'
pathnomi = '/content/drive/My Drive/gephi/nomi_italiani.txt' 
pathcognomi = '/content/drive/My Drive/gephi/cognomi.txt' 
pathstati = '/content/drive/My Drive/gephi/stati.txt' 
pathcomuni = '/content/drive/My Drive/gephi/comuni.csv'
import os
#CREIAMO UNA LISTA CONTENENTE TUTTI I NOMI ITALIANI
var_nomi_italiani = open(pathnomi,'r').read().split('\n')
for i in range (0, len(var_nomi_italiani)):
  check = var_nomi_italiani[i]
  var_nomi_italiani[i] = check.capitalize()
print("Nomi italiani")
print(var_nomi_italiani)
print("")
#CREIAMO UNA LISTA CONTENENTE TUTTI I COGNOMI ITALIANI
var_cognomi = open(pathcognomi,'r').read().split('\n')
print("Cognomi italiani")
print(var_cognomi)
print("")
#CREIAMO UNA LISTA CONTENENTE TUTTI GLI STATI IN ITALIANO
var_stati = open(pathstati,'r').read().split('\t')
var_stati.pop(0)
var_stati.pop(0)
var_stati.pop(0)
for i in range(0,len(var_stati)):
  if i % 2 != 0:
    var_stati[i]=""
var_stati_clean=[]
[var_stati_clean.append(item) for item in var_stati if item not in var_stati_clean]
var_stati_clean.pop(1)
print("Stati in italiani puliti")
print(var_stati_clean)
print("")

#CREIAMO UNA LISTA CONTENENTE TUTTI I COMUNI ITALIANI
df1 = pd.read_csv('/content/drive/My Drive/gephi/comuni.csv', encoding = "ISO-8859-1")
list_comuni = df1.values.tolist()
var_comuni = []
for elem in list_comuni:
    var_comuni.extend(elem)
print("Tutti i comuni italiani")
print(var_comuni)
print("")

#CREIAMO UNA LISTA CONTENENTE TUTTI NOMI PROPRI IN ITALIANO
lista_ita = var_cognomi + var_nomi_italiani + var_comuni + var_stati_clean
lista_ita.append("Know") 
print(f'In tutto i nomi prori in italiano sono -->  {len(lista_ita)}')

Blocco che si occupa della pulizia del dataset

In [ ]:
import nltk
import pandas as pd
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.snowball import ItalianStemmer
import re
#Scarichiamo le stopword e importiamo solo quelle italiane
nltk.download('punkt')
nltk.download('stopwords')
stops = set(stopwords.words("italian"))
#Stemming per rimuovere le ultime lettere (Se esse si riferiscono a plurale, cognugazione dei verbi e genere)
stemming = SnowballStemmer("italian", ignore_stopwords=False)

def apply_cleaning_function_to_list(X):
    cleaned_X = []
    for element in X:
        cleaned_X.append(clean_text(element))
    return cleaned_X

def clean_text(raw_text):

  text = raw_text 

#VARI CONTROLLI PER LE LETTERE MAIUSCOLO DOPO I SEGNI DI PUNTEGGIATURA. METTE MINUSCOLA LA LETTERA SUCCESSIVA ALLA PUNTEGGIATURA. (Conte. Rubrica --> Conte rubrica)
  lista = list(text)
  for i in range(0,len(lista)):
    if(lista[i]=='"' and i+1 < len(lista) or lista[i]=='“' and i+1 < len(lista) or lista[i]=="'" and i+1 < len(lista)):
      if lista[i+1] not in lista_ita:
        lista[i+1] = lista[i+1].lower()

    if(lista[i]=='.'  and i+1 < len(lista)):
      if(lista[i+1]==' ' and i+2 < len(lista)):
        if lista[i+2] not in lista_ita:
          lista[i+2] = lista[i+2].lower()
    
    if(lista[i]=='?'  and i+1 < len(lista)):
      if(lista[i+1]==' ' and i+2 < len(lista)):
        if lista[i+2] not in lista_ita:
          lista[i+2] = lista[i+2].lower()
    
    if(lista[i]=='!'  and i+1 < len(lista)):
      if(lista[i+1]==' ' and i+2 < len(lista)):
        if lista[i+2] not in lista_ita :
          lista[i+2] = lista[i+2].lower()
          
  text = "".join(lista)

  #TOCKEN PER DIVIDERE LE PAROLE DEI TITOLI
  tokens = nltk.word_tokenize(text, language='italian')
  #CONVERSIONE IN MINUSCOLO DELLE PAROLE DI OGNI TITOLO
  if tokens[0] not in lista_ita:
      tokens[0] = tokens[0].lower()

  #ESCLUDIAMO LA PAROLA "nan" POICHE' GENERA ERRORE
  token_words = [w for w in tokens if w != "nan"] 
  
  text_nostopword = token_words
  #COMPOSIZIONE NomeMaiuscolo
  for i in range (0,len(text_nostopword)):
    check = text_nostopword[i]
    check3 = ""
    if(check.lower() not in stops):
      if check.islower() :
        text_nostopword[i] = stemming.stem(check)
      elif (text_nostopword[(i)]).isalpha():
        if i+1 < len(text_nostopword):
          if not((text_nostopword[(i+1)]).isalpha()):
            continue
          check2 = text_nostopword[(i+1)]
          if check2.istitle():
            if check2.lower() not in stops:
              text_nostopword[i] = check + check2
              text_nostopword[i+1]= ""
          if i+2 < len(text_nostopword):
            if not((text_nostopword[(i+2)]).isalpha()):
              continue
            check3 = text_nostopword[(i+2)]
            if check2.istitle() and check2.lower() not in stops and check3.istitle() and check3.lower() not in stops:
              #unisci le due parole
              text_nostopword[i] = check + check2 + check3
              text_nostopword[i+1]= ""
              text_nostopword[i+2]= ""


  text_nostopword = [w for w in text_nostopword if  w.isalpha() ] 
  #RIMUOVIAMO LE STOPWORDS
  text_nostopwords = [w for w in text_nostopword if not w.lower() in stops and len(w)>2 ]

 
  #REJOIN DELLE PAROLE
  joined_words = ( " ".join(text_nostopwords))

  #RESUB DELLE VARIE PAROLE SIMILI TRA LORO 
  joined_words = re.sub(r"coronavirus", "Coronavirus", joined_words)
  joined_words = re.sub(r"covid", "Coronavirus", joined_words)
  joined_words = re.sub(r"Covid", "Coronavirus", joined_words)
  joined_words = re.sub(r"nelmercatodiwuhan","mercato Wuhan", joined_words)
  joined_words = re.sub(r"cittàinquaranten","citt quaranten", joined_words)
  joined_words = re.sub(r"doveènatoilvirus","virus", joined_words) 
  #GLI ARTICOLI SENZA TITOLO VERRANNO CANCELLATI
  joined_words = re.sub(r"titol", "", joined_words)
  
  testopulito = joined_words 
  
  return testopulito


#CARICO IL CSV DA PULIRE
imdb = pd.read_csv(path)

#CREO LA LISTA DEI TITOLI
text_to_clean = list(imdb['title'])

#PULIZIA DEL CSV
for i in range (0, len(text_to_clean)):
  text_to_clean[i] = str(text_to_clean[i])
cleaned_text = apply_cleaning_function_to_list(text_to_clean)

#STAMPIAMO TITOLI PULITI E NON (PER AVERE UN ESEMPIO)
print ('Original text:',text_to_clean[5])
print ('\nCleaned text:', cleaned_text[5])

#INSERIRE TUTTI I DATI PULITI NEL DATAFREME 
imdb['title'] = cleaned_text
imdb.to_csv('imdb.csv', index=False) #True per avere id prima della frase

Blocco che permette di creare l'insieme dei nodi e di salvarli all'interno di un CSV.

In [ ]:
import csv
with open('nodes_table.csv', 'w',newline='') as f:
  with open('imdb.csv') as csv_file:
    nodi = []
    csv_reader = csv.reader(csv_file, delimiter=' ')
    thewriter = csv.writer(f) 
    thewriter.writerow(['Id','Label']) 
    for row in csv_reader:
        for i in range (0,len(row)):
          if row[i] not in nodi and len(row[i])>2:
            nodi.append(row[i])
    nodi.sort()
    print(nodi)
    for i in range(0,len(nodi)):
      thewriter.writerow([i,nodi[i]])
print(len(nodi))

Blocco che permette di creare l'insieme totale degli archi.

In [ ]:
import csv
with open('edges_table.csv', 'w',newline='') as f:
  with open('imdb.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=' ')
    lista_edge =[]
    line_count = 0
    thewriter = csv.writer(f) 
    thewriter.writerow(['Source','Target']) 
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            line_count += 1
        else:
            for i in range (0,len(row)):
              for c in range (i,len(row)):
                if c != i  and len(row[i])>2 and len (row[c])>2:
                  thewriter.writerow([nodi.index(row[i]),nodi.index(row[c])])
                  edge1 = [nodi.index(row[i]),nodi.index(row[c])]
                  lista_edge.append(edge1)
            line_count += 1
    print(f'Processed {line_count} lines.')
    print(lista_edge)
    print("GLI ARCHI TOTALI SONO : ",len(lista_edge))

Blocco per creare la lista che contiene gli archi unici.

In [ ]:
import progressbar
bar = progressbar.ProgressBar(maxval=len(lista_edge), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

lista_single = []
bar.start()

for i in range(0,len(lista_edge)):

  bar.update(i+1)

  sourcei = lista_edge[i][0]
  targeti = lista_edge[i][1]
  inverso = [targeti, sourcei]

  if lista_edge[i] not in lista_single and inverso not in lista_single:
    lista_single.append(lista_edge[i])
  
bar.finish()
print(lista_single)
print("GLI ARCHI UNICI SONO : ",len(lista_single))

Blocco che permette di creare la lista con gli archi pesati 

In [ ]:
import progressbar
bar = progressbar.ProgressBar(maxval=len(lista_single), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])

lista_pesata = []

bar.start()
for i in range(0,len(lista_single)):
  
  bar.update(i+1)

  sourcei = lista_single[i][0]
  targeti = lista_single[i][1]
  inverso = [targeti, sourcei]

  peso1 = lista_edge.count(lista_single[i])
  peso2 = lista_edge.count(inverso)

  peso = peso1 + peso2
  lista_pesata.append(peso)

bar.finish()
print(lista_pesata)
print("GLI ARCHI PESATI SONO : ",len(lista_pesata))

Blocco per copiare la lista degli archi all'interno del CSV

In [ ]:
with open('edges_table.csv', 'w',newline='') as f:

  thewriter = csv.writer(f) 
  thewriter.writerow(['Source','Target','Weight'])

  for i in range(0,len(lista_pesata)):
    thewriter.writerow([lista_single[i][0],lista_single[i][1],lista_pesata[i]])
